LUAD DATASET CODE FOR GITHUB AND FINAL ANALYSIS WORKFLOW

In [ ]:
library(MultiAssayExperiment)
library(curatedTCGAData)
library(TCGAutils)
library(UpSetR)
library(DESeq2)
library(M3C)
library(MOFA2)
library(data.table)
library(tidyverse)

#curatedTCGAData(diseaseCode = "LUAD", assays = "*", dry.run = TRUE, version = "1.1.38")

# Todo: Rename to something understandable
luad.mae <- curatedTCGAData(
  diseaseCode = "LUAD",
  assays = c(
    "RPPAArray", "Mutation",
    "RNASeq2GeneNorm", # here perhaps the raw RSEM counts
    "GISTIC_ThresholdedByGene"
  ),
  dry.run = FALSE,
  version = "1.1.38"
)

# Draw diagram
upsetSamples(luad.mae)

rag <- "LUAD_Mutation-20160128"
genome(luad.mae[[rag]]) <- translateBuild(genome(luad.mae[[rag]]))
seqlevelsStyle(luad.mae[[rag]]) <- "UCSC"
genome(luad.mae[[rag]])

# Convert experiment
luad.updated <- qreduceTCGA(luad.mae, keep.assay = FALSE) # dont keep the original muts
sampleTables(luad.updated)

tums <- TCGAsampleSelect(colnames(luad.updated), "01")
luad.processed <- luad.updated[, tums, ]

luad.final <- mergeReplicates(intersectColumns(luad.processed))


# normalize and filter accordingly any included molecular layers
cnas.matrix <- assay(luad.final[[1L]])
snvs.matrix <- assay(luad.final[[4L]])
protein.dat <- assay(luad.final[[3L]])

rna.seq <- getWithColData(luad.final, 2L)
count.dat <- assay(rna.seq)
pheno.dat <- colData(rna.seq)

# If you have a matrix of RSEM values, the way to use the above workflow would be to import counts and lengths and construct a dds object from these. The counts (not scaled) can be rounded to integers and passed to DESeqDataSetFromMatrix.

dds <- DESeqDataSetFromMatrix(
  countData = round(count.dat),
  colData = pheno.dat,
  design = ~1
)

# optional filtering just to remove genes with 0 counts in all samples
# keep <- rowSums(counts(dds)) > 1
# dds <- dds[keep,]
# nrow(dds)

dds.norm <- vst(dds) # re-check

# continue here with non-specific filtering implementation

NotExpressed <- apply(assay(dds.norm), MARGIN = 2, function(z) {
  dens <- density(z)
  expr.cut <- dens$x[which.max(dens$y)]
  return(z < expr.cut)
})

# with the above code chunk you'll get a boolean matrix with the columns corresponding to the rna-seq samples, the rows to the genes and the elements being TRUE if the expression value is lower than the cut-off value for that arrays, and FALSE otherwise:

expr.ps <- rowSums(!NotExpressed) > (ncol(NotExpressed) / 2)
dds.filtered <- dds.norm[expr.ps, ]

# so you could filter your dataset to keep all those "genes" that have an expression value larger than the cut-off value you defined above (i.e. are presumably expressed) in at least half of the samples of your data set.

# extra filtering regarding variance

filtered_results <- featurefilter(assay(dds.filtered), percentile = 40, method = "MAD", topN = 10000)

filtered.exp.final <- as.matrix(filtered_results$filtered_data)

keep.mut <- rownames(assay(dds.filtered))

keep.mut.final <- rownames(filtered.exp.final)
# then further keep the overlapping genes with the CNAs and the somatic point mutations

common.cnas <- intersect(rownames(cnas.matrix), keep.mut.final)
common.muts <- intersect(rownames(snvs.matrix), keep.mut.final)

cnas.sel <- cnas.matrix[common.cnas, ] # for example there is TP53 in CNAs
mut.sel <- snvs.matrix[common.muts, ] # also but with 0s

protein.dat.fil <- na.omit(protein.dat)

###############################################################################
###############################################################################
# Change the mutational matrix table of genes as selected clinical covariates
# count(Gene_Symbol,XTENS_name,sort = T)

# ------------------------- TODO: NO DATA IN LAYER 4! ------------------------
to.plot <- data.table(
  mutation = rownames(mut.sel),
  frequency = rowMeans(mut.sel, na.rm = T)
) # perhaps filter based on the total frequency-for example, < than 2%



# keep the top 100 most frequently mutated samples

freq.mut.wes <- to.plot[order(to.plot$frequency, decreasing = T), ][1:100]
# write_tsv(freq.mut.wes,"TOP100.Freq.Mut.LUAD.MAE.txt")

ggplot(freq.mut.wes, aes(x = reorder(mutation, -frequency), y = frequency)) +
  geom_bar(stat = "identity") +
  theme_classic() +
  labs(x = "", y = "Frequency") +
  theme(
    axis.text.x = element_text(colour = "black", size = rel(0.7), angle = 90, hjust = 1, vjust = 0.5),
  ) # needs much smaller number of genes to work for the x-axis


# Then the final step for the somatic mutations is to perform a simple overlap analysis between the aforementioned top100 genes with the list of COSMIC Cancer Gene Census:
# https://cancer.sanger.ac.uk/census

mydir <- getwd() # folder or dir where the COSMIC txt file has been downloaded
setwd(mydir)

# read the cancer census genes list from COSMIC
cosmic.census.broad <- read_tsv("data/Census_allMon Dec 14 12_02_03 2020.tsv")
cosmic.census.genes <- cosmic.census.broad$`Gene Symbol`

sel.mut <- intersect(freq.mut.wes$mutation, cosmic.census.genes) # with top100

mut.sel.final <- mut.sel[sel.mut, ]

mutations.dat <- data.frame(t(mut.sel.final)) # 181 samples and change the names

# in order to merge the clinical metadata with the selected mutations, we need to retreive the full barcodes from the TCGA sample names as below:

fullBarcodes <- rownames(mutations.dat)
sample_ids <- TCGAbarcode(fullBarcodes)
rownames(mutations.dat) <- sample_ids

# the check and align that the sample names have the same order between the selected mutations and the clinical data, in order to create a final data frame of available clinical metadata

# TCGA clinical phenodata comments: These are artifacts of the merged clinical datasets. For each cancer the Clinical Supplement data with the Clinical_Pick_Tier1 data have been include. See here for more information: https://broadinstitute.atlassian.net/wiki/spaces/GDAC/pages/844334346/Documentation#Documentation-ClinicalPipelineClinicalDataProcessing


# general comment for the some "duplicated" clinical variables: These variables come from the Broad GDAC Firehose so I would look for the documentation and try to make out why these variables are different. They could have been taken at different time points. Generally it's preferable to have less NA values but the provenance of those variables have to be taken into account as they're not necessarily "duplicated".

clin.dat <- as.data.frame(colData(luad.final))

clinical.filtered <- clin.dat %>%
  select(c(
    patientID, years_to_birth, vital_status, pathologic_stage, pathology_T_stage, pathology_N_stage,
    gender, radiation_therapy, histological_type,
    patient.clinical_cqcf.anatomic_neoplasm_subdivision, patient.tobacco_smoking_history,
    days_to_death, days_to_last_followup, days_to_last_known_alive,
    expression_subtype
  ))


staget <- sub("[abcd]", "", sub("t", "", clinical.filtered$pathology_T_stage))
staget <- suppressWarnings(as.integer(staget))
clinical.filtered$pathology_T_stage <- staget


identical(rownames(clinical.filtered), rownames(mutations.dat))

final.clin.dat <- bind_cols(clinical.filtered, mutations.dat)

colData(luad.final) <- S4Vectors::DataFrame(final.clin.dat) # # where DF is the filtered / cleaned clinical `S4Vectors::DataFrame` with only the "interesting" clinical variables. It only works with a `DataFrame`'value'. It currently does not work with `data.frame`s or `tibble`s

luad2 <- c(luad.final, list(RNAseq = filtered.exp.final), mapFrom = "LUAD_RNASeq2GeneNorm-20160128") # for a separate assay to be added and mapped from a previous one-this for the rna-seq in order not to loss the sampleNames

sel.mae <- luad2[, , c(1, 3, 5)]
# use the single bracket method ([) in conjunction with a List/list or ExperimentList of the same length as the subsetting vector-can handle multiple replacements.

sel.mae[, , c(1, 2)] <- list(CNV = cnas.sel, RPPA = protein.dat.fil) # for the expression data will not work if used getWithColData, as it takes only the TCGA barcodes


##################################################################################
# Modify some of the assays to prepare the final input for creating the last MultiAssayExperiment

e <- assays(sel.mae)
e[[2]] %>% as.matrix() -> e[[2]]
# Rename features so that there are no duplicated gene names for different omic layers
paste("gistic", e[[1]] %>% rownames(), sep = "_") -> rownames(e[[1]])
paste("rppa", e[[2]] %>% rownames(), sep = "_") -> rownames(e[[2]])
paste("rna", e[[3]] %>% rownames(), sep = "_") -> rownames(e[[3]])

mae <- MultiAssayExperiment(experiments = e, colData = colData(sel.mae), sampleMap = sampleMap(sel.mae))

RUN THE FINAL MULTI-OMICS DATASET WITH MOFA2 computational workflow

In [ ]:
MOFAobject <- create_mofa_from_MultiAssayExperiment(mae, extract_metadata = T)

plot_data_overview(MOFAobject)

data_opts <- get_default_data_options(MOFAobject)
data_opts

model_opts <- get_default_model_options(MOFAobject) # likelihood per view (options are “gaussian”, “poisson”, “bernoulli”). By default they are inferred automatically.
model_opts$num_factors <- 15

train_opts <- get_default_training_options(MOFAobject)
#train_opts$convergence_mode <- "medium" # For exploration, the fast mode is good enough.
train_opts$convergence_mode <- "fast" # For exploration, the fast mode is good enough.

# train the MOFA object

MOFAobject <- prepare_mofa(MOFAobject,
  data_options = data_opts,
  model_options = model_opts,
  training_options = train_opts
)


outfile <- file.path(getwd(), "caseStudy.Finalmodel.LUAD.hdf5")
MOFAobject.trained <- run_mofa(MOFAobject, outfile, use_basilisk = TRUE)

# Load precomputed model

# http://www.ebi.ac.uk/shiny/mofa/

model <- load_model("caseStudy.Finalmodel.LUAD.hdf5")
views_names(model) <- c("CNV", "RPPAArray", "RNASeq")


# EDA PHASE_1-Correlation between factors-A good sanity check is to verify that the Factors are largely uncorrelated. In MOFA there are no orthogonality constraints such as in Principal Component Analysis, but if there is a lot of correlation between Factors this suggests a poor model fit. Reasons? Perhaps you used too many factors or perhaps the normalization is not adequate.

plot_factor_cor(MOFAobject.trained)

# EDA PHASE_2-Variance decomposition by Factor-The most important insights that MOFA generates is the variance decomposition analysis. This plot shows the percentage of variance explained by each factor across each data modality (and group, if provided). It summarizes the sources of variation from a complex heterogeneous data set in a single figure.

plot_variance_explained(MOFAobject.trained, max_r2 = 10)
plot_variance_explained(MOFAobject.trained, plot_total = T)[[2]]
model@cache$variance_explained$r2_total[[1]]

#correlate_factors_with_covariates(MOFAobject.trained,
#  covariates = c(
#    "AKAP9", "ATM", "BCORL1", "CNTNAP2", "COL3A1", "EGFR", "EPHA3", "FAT4", "MET", "NBEA", "NF1", "PDGFRA", "SETBP1", "expression_subtype", "pathology_T_stage",
#    "patient.tobacco_smoking_history"
#  ),
#  plot = "log_pval"
#)

# Plot the individual top weights on selected Factors regarding specific layers:


plot_weights(model,
  view = "CNV",
  factor = 3,
  nfeatures = 20, # Top number of features to highlight
  scale = T # Scale weights from -1 to 1
)

plot_top_weights(model,
  view = "RNASeq",
  factor = 1, # c(1,2)
  nfeatures = 20,
  scale = T
)

plot_top_weights(model,
  view = "RPPAArray",
  factor = 1,
  nfeatures = 20,
  scale = T
)

plot_factor(MOFAobject.trained,
  factor = 1:3,
  color_by = "expression_subtype",
  # shape_by = "ATM" <-- Not supported
)

atm_plot <- plot_factor(
  model, 
  factor = 2, 
  # color_by = "ATM", <-- Not supported
  dodge = T, 
  add_violin = T, 
  add_boxplot = T
)

p2 <- atm_plot + ggpubr::stat_compare_means()
plot(p2) # add statistical group comparison significance p-value


plot_factor(
  model, 
  factor = 1, 
  # color_by = "expression_subtype",  <-- Not supported
  dodge = T, 
  add_violin = T, 
  add_boxplot = T
)


plot_data_scatter(model,
  view = "RNAseq",
  factor = 1,
  features = 4,
  sign = "positive",
  # color_by = "expression_subtype" <-- Not supported
) + labs(y = "RNA expression")


# get the weights of selected views and factors to perform external functional enrichment analysis if not following the GSEA implementation in MOFA2

sel.weights <- get_weights(model, views = "all", factors = "all", as.data.frame = TRUE)

# Perhaps do an extra scaling of the retrieved weights to select a specific number for downstream enrichment analysis-also it is doable that the weights value per factor can be scaled. While it is very arbitrary to select features based on a threshold, it might provide a basis for further feature selection methodologies based on the distribution of the weights. Also consider splitting the features by positive/negative weight

For downstream enrichment analysis of selected factors/features

In [ ]:
library(msigdbr)
library(data.table)

# if not loaded from above analysis chunks

model <- load_model("caseStudy.Finalmodel.LUAD.hdf5")
views_names(model) <- c("CNV", "RPPAArray", "RNASeq")

matrix.please <- function(x) {
  m <- as.matrix(x[, -1])
  rownames(m) <- x[[1]]
  m
}

# download specific gene-sets from MSigDB-here we select REACTOME pathways

msgidb.matrix <- msigdbr(
  species = "Homo sapiens",
  category = "C2",
  subcategory = "REACTOME"
) %>%
  as.data.table() %>%
  .[, id := 1] %>%
  dcast(gs_name ~ gene_symbol, value.var = "id", fill = 0) %>%
  matrix.please()

# example for performing analysis for the gene expression data-rename accordingly the relative features

colnames(msgidb.matrix) <- str_c("rna_", colnames(msgidb.matrix))

# GSEA on positive weights
gsea.positive <- run_enrichment(model,
  feature.sets = msgidb.matrix,
  view = "RNASeq", # change perhaps factors
  sign = "positive"
)

# GSEA on negative weights
gsea.negative <- run_enrichment(model,
  feature.sets = msgidb.matrix,
  view = "RNASeq",
  sign = "negative"
)

plot_enrichment(gsea.positive, factor = 1, max.pathways = 15)
plot_enrichment(gsea.negative, factor = 1, max.pathways = 15)

plot_enrichment_detailed(gsea.positive,
  factor = 1,
  max.genes = 10,
  max.pathways = 5
)

DOWNSTREAM MULTI-OMICS CLUSTERING ANALYSIS

In [ ]:


# modify the cluster_samples function to extract the numeric latent factors from the trained model, in order to be used as an input to kmeans clustering or any other clustering or dimensionality reduction approach

get_mofa_factors <- function(object, factors = "all", ...) {
  Z <- get_factors(object, factors = factors)
  if (is(Z, "list")) {
    Z <- do.call(rbind, Z)
  }
  N <- nrow(Z)
  haveAllZ <- apply(Z, 1, function(x) all(!is.na(x)))
  if (!all(haveAllZ)) {
    warning(paste("Removing", sum(!haveAllZ), "samples with missing values on at least one factor"))
  }
  Z <- Z[haveAllZ, ]
  return(Z)
}

mfact <- get_mofa_factors(object = model, factors = "all")

# https://scikit-learn.org/stable/modules/clustering.html
# https://www.r-bloggers.com/2017/02/finding-optimal-number-of-clusters/

# One approach---The elbow method looks at the percentage of variance explained as a function of the number of clusters:

set.seed(123) # nstart = 25
# Compute and plot wss for k = 2 to k = 6.
k.max <- 6
data <- mfact
wss <- sapply(
  1:k.max,
  function(k) {
    kmeans(data, k, nstart = 50, iter.max = 15)$tot.withinss
  }
)
wss
plot(1:k.max, wss,
  type = "b", pch = 19, frame = FALSE,
  xlab = "Number of clusters K",
  ylab = "Total within-clusters sum of squares"
)


# We will continue with Monte Carlo Reference-based Consensus Clustering
library(M3C)

dt <- as.data.frame(t(data))
# various options for objective function,iterations, clustering algorithm
res <- M3C(dt, iters = 100, clusteralg = "km", fsize = 8, lthick = 1, dotsize = 1.25, repsref = 250, repsreal = 250) # keep this

myresults <- res$realdataresults[[3]]$ordered_annotation
data <- res$realdataresults[[3]]$ordered_data

# fetch input data for each view-each one as a separate list

input_cnv <- get_data(model, views = "CNV")
mat_cnv <- as.matrix(input_cnv$CNV$group1)

input_rnaseq <- get_data(model, views = "RNASeq")
mat_rnaseq <- as.matrix(input_rnaseq$RNASeq$group1)

input_rppa <- get_data(model, views = "RPPAArray")
mat_rppa <- as.matrix(input_rppa$RPPAArray$group1)

total.clin.dat <- model@samples_metadata

# extract the top features for each data layer:

sel.weights.rna <- get_weights(model, views = "RNASeq", factors = c(1, 2, 5), as.data.frame = TRUE, scale = T)
feat.sel.rna <- sel.weights.rna %>%
  group_by(factor) %>%
  top_n(n = 5, wt = abs(value))
sel.rna.names <- as.character(feat.sel.rna$feature)


sel.weights.rppa <- get_weights(model, views = "RPPAArray", factors = 1, as.data.frame = TRUE, scale = T)
feat.sel.rppa <- sel.weights.rppa %>% top_n(n = 10, wt = abs(value))
sel.rppa.names <- as.character(feat.sel.rppa$feature)


sel.weights.cna <- get_weights(model, views = "CNV", factors = c(3, 4, 6), as.data.frame = TRUE, scale = T)
feat.sel.cna <- sel.weights.cna %>%
  group_by(factor) %>%
  top_n(n = 5, wt = abs(value))
ff <- feat.sel.cna[c(1, 2, 3, 4, 5, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22), ] # manually extract due to the distinct values of CNAs, various features get similar scores

sel.cna.names <- as.character(ff$feature)

sel.mat.cnv <- mat_cnv[sel.cna.names, ]
rownames(sel.mat.cnv) <- str_remove(rownames(sel.mat.cnv), "gistic_")

sel.mat.rnaseq <- mat_rnaseq[sel.rna.names, ]
rownames(sel.mat.rnaseq) <- str_remove(rownames(sel.mat.rnaseq), "rna_")

sel.mat.rppa <- mat_rppa[sel.rppa.names, ]
rownames(sel.mat.rppa) <- str_remove(rownames(sel.mat.rppa), "rppa_")

In [ ]:
# Create the final Complex Heatmaps

library(ComplexHeatmap)
library(circlize)
library(dplyr)
library(RColorBrewer)
library(corrplot)

pdat <- model@samples_metadata
rownames(pdat) <- pdat$sample

pdat_sel <- pdat[, colnames(pdat) %in% c(
  "ATM", "EGFR", "MET", "NF1", "NBEA", "NF1",
  "expression_subtype", "pathology_T_stage", "pathology_N_stage", "gender",
  "patient.tobacco_smoking_history"
)]

clust.member <- as.data.frame(res$realdataresults[[3]]$assignments) # the cluster assignments for each sample, keeping the order as the input

pdat_sel <- pdat_sel %>% droplevels()
identical(rownames(clust.member), rownames(pdat_sel))

pdat_sel$groups_km_LF <- as.factor(clust.member$`res$realdataresults[[3]]$assignments`)

pdat_sel <- pdat_sel[order(pdat_sel$groups_km_LF), ] # order the clinical data based on the actual order of the clusters

# scaling only for the rna-seq data

scaled_mat <- t(scale(t(sel.mat.rnaseq)))

hist(scaled_mat) # shows the distribution/range of z-scores to define the boundaries of the colors below

pdat_sel <- pdat_sel %>%
  mutate(
    patient.tobacco_smoking_history =
      case_when(
        patient.tobacco_smoking_history == "current reformed smoker for < or = 15 years" ~
        "Active or former",
        patient.tobacco_smoking_history == "current reformed smoker for > 15 years" ~ "Active or former",
        patient.tobacco_smoking_history == "lifelong non-smoker" ~ "Never-smoker",
        patient.tobacco_smoking_history == "current smoker" ~ "Active or former",
        TRUE ~ patient.tobacco_smoking_history
      )
  ) %>%
  dplyr::rename(Smoking_Status = patient.tobacco_smoking_history)

# COL3A1, AKAP9, gender and expression_subtype significant based on the following chi-square test for independence

chifit <- suppressWarnings(chisq.test(table(pdat_sel[c("groups_km_LF", "gender")])))
contrib <- 100 * chifit$residuals^2 / chifit$statistic
corrplot(contrib, is.cor = FALSE)

# reorder columns for each numeric dataset
rna_scaled_order <- scaled_mat[, rownames(pdat_sel)]
mat_cnv_order <- sel.mat.cnv[, rownames(pdat_sel)]
mat_rppa_order <- sel.mat.rppa[, rownames(pdat_sel)]

# below choose between different color palettes for each layer

f <- colorRamp2(c(-2, 0, 2), c("green", "black", "red"))

# f1 <- colorRamp2(c(-2, 0, 2), c("blue", "#EEEEEE", "red"), space = "sRGB")
# col_cna_fun <- f1(at)

f2 <- colorRamp2(seq(min(mat_cnv_order), max(mat_cnv_order), length = 3), c("blue", "#EEEEEE", "red"), space = "RGB")

f3 <- colorRamp2(c(-2, 0, 2), c("blue", "white", "yellow"))

col_fun_1 <- colorRamp2(c(-2, 0, 2), c("#377EB8", "white", "#E41A1C"))

myCol <- colorRampPalette(c("royalblue", "white", "red3"))(100)

abs_x <- abs(mat_cnv_order)
q <- quantile(abs_x[abs > 0], 0.95)
col_fun <- colorRamp2(c(-q, 0, q), c("#377EB8", "white", "#E41A1C"))

# also to map colors to 4 intervals you could do the following:
cf2 <- colorRamp2(c(-2, -1, 0, 1, 2), c("blue", "navy", "white", "orange", "red"))

ha_test <- HeatmapAnnotation(df = pdat_sel) # take and run by default all columns

# define manually the colors for each selected clinical covariate

mutation_col <- structure(
  names = c("MUT", "WT"),
  c("black", "white")
)

ha <- HeatmapAnnotation(
  AKAP9_mut = ifelse(pdat_sel[[1]] == 1, "MUT", "WT"),
  EGFR_mut = ifelse(pdat_sel[[4]] == 1, "MUT", "WT"),
  COL3A1_mut = ifelse(pdat_sel[[3]] == 1, "MUT", "WT"),
  expr_subtype = pdat_sel[[5]],
  gender = pdat_sel[[6]],
  N_Stage = pdat_sel[[7]],
  T_Stage = pdat_sel[[8]],
  Smoking_Status = pdat_sel[[9]],
  Clusters_km_LF = pdat_sel[[10]],
  col = list(
    expr_subtype = structure(names = c("TRU", "prox.-prolif.", "prox.-inflam"), c("#377EB8", "#FFFF33", "#FF7F00")),
    Clusters_km_LF = structure(
      names = c("1", "2", "3"),
      c("black", "red", "blue")
    ),
    T_Stage = c("1" = "#377EB8", "2" = "blue", "3" = "#FF7F00", "4" = "#FF0000"),
    N_Stage = structure(
      names = c("n0", "n1", "n2", "n3", "nx"),
      c("yellow", "red", "blue", "brown", "grey")
    ),
    Smoking_Status = structure(names = c("Active or former", "Never-smoker", "NA"), c("#ca0020", "black", "#f7f7f7")),
    gender = structure(names = c("female", "male"), c("#7b3294", "#008837")),
    AKAP9_mut = mutation_col,
    EGFR_mut = mutation_col,
    COL3A1_mut = mutation_col
  ),
  na_col = "grey", border = TRUE, # show_legend
  show_annotation_name = FALSE
) # for the names of the annotation legends

ht1 <- Heatmap(rna_scaled_order, name = "Scaled_gene_expression", cluster_columns = FALSE, top_annotation = ha, show_column_names = FALSE, col = col_fun_1, row_names_gp = gpar(fontsize = 9), row_title = "RNASeq", show_row_dend = FALSE)

ht2 <- Heatmap(mat_cnv_order,
  cluster_columns = FALSE, name = "CNV",
  show_column_names = FALSE, col = f1, row_names_gp = gpar(fontsize = 9), row_title = "CNA", heatmap_legend_param = list(color_bar = "discrete"), show_row_dend = FALSE
)

ht3 <- Heatmap(mat_rppa_order,
  cluster_columns = FALSE, "Protein_Expression",
  show_column_names = FALSE, col = f3, row_names_gp = gpar(fontsize = 9), row_title = "RPPA", show_row_dend = FALSE
)

ht_list <- ht1 %v% ht2 %v% ht3

tiff("Complex.Heatmaps.MOFA.tiff", width = 12, height = 8, units = "in", res = 300)

draw(ht_list,
  row_title = "Multi-omics selected MOFA features",
  annotation_legend_side = "left", heatmap_legend_side = "left"
) # right

annotation_titles <- c(
  AKAP9_mut = "AKAP9 mutation",
  EGFR_mut = "EGFR Mutation",
  COL3A1_mut = "COL3A1 mutation",
  expr_subtype = "Expression Subtypes",
  gender = "Gender",
  N_Stage = "Node Stage",
  T_Stage = "Tumor Stage",
  Smoking_Status = "Smoking History",
  Clusters_km_LF = "Multi-omics clusters"
)
for (an in names(annotation_titles)) {
  decorate_annotation(an, {
    grid.text(annotation_titles[an], unit(-2, "mm"), just = "right")
    grid.rect(gp = gpar(fill = NA, col = "black"))
  })
}


dev.off()

Survival analysis

In [ ]:
library(survival)
library(survminer)

SurvObject <- Surv(model@samples_metadata$days_to_death, model@samples_metadata$vital_status)

Z <- get_factors(model)[[1]]
fit <- coxph(SurvObject ~ Z)
fit

s <- summary(fit)
coef <- s[["coefficients"]]

df <- data.frame(
  factor = factor(rownames(coef), levels = rev(rownames(coef))),
  p      = coef[, "Pr(>|z|)"],
  coef   = coef[, "exp(coef)"],
  lower  = s[["conf.int"]][, "lower .95"],
  higher = s[["conf.int"]][, "upper .95"]
)

ggplot(df, aes(x = factor, y = coef, ymin = lower, ymax = higher)) +
  geom_pointrange(col = "#619CFF") +
  coord_flip() +
  scale_x_discrete() +
  labs(y = "Hazard Ratio", x = "") +
  geom_hline(aes(yintercept = 1), linetype = "dotted") +
  theme_bw()

df <- data.frame(
  time = SurvObject[, 1],
  event = SurvObject[, 2], Z1 = Z[, 1]
)
cut <- surv_cutpoint(df, variables = "Z1")
df$FactorCluster <- df$Z1 > cut$cutpoint$cutpoint
fit <- survfit(Surv(time, event) ~ FactorCluster, df)

ggsurvplot(fit,
  data = df,
  conf.int = TRUE, pval = TRUE,
  fun = function(y) y * 100,
  legend = "top", legend.labs = c(paste("low LF 1"), paste("high LF 1")),
  xlab = "Overall Survival", ylab = "Survival probability (%)", title = "Factor 1"
)$plot